In [1]:
import os
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")  
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_TRACING_V2=True

In [ ]:
# Helper function for printing docs


def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

# Step-1: Document Loading

In [2]:
from langchain.document_loaders import PyPDFLoader

In [10]:

loader = PyPDFLoader("data/EMPLOYEE_AGREEMENT.pdf")
pages = loader.load()
print(len(pages))

13


In [11]:
full_text =""
for page in pages:
  full_text += page.page_content

print("Pages", len(pages))
print("Lines" , len(full_text.split("\n")))
print("Words" , len(full_text.split(" ")))
print("Charecters", len(full_text))

Pages 13
Lines 369
Words 4481
Charecters 29817


# Step-2:Split the data into Chunks

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
chunks = text_splitter.split_documents(pages)
print(len(chunks))

133


In [14]:
print(chunks[0])

page_content='EX-10.1 3 smtp_ex10z1.htm EMPLOYEE AGREEMENT\nEXHIBIT  10.1\nEMPLOYEE AGREEMENT\nTHIS EMPLOYEE AGREEMENT  made as of September___, 2014, by and between SharpSpring,\nInc., a Delaware corporation (the “Company ”), whose principal place of business is at 802 NW 5th Avenue,' metadata={'source': 'data/EMPLOYEE_AGREEMENT.pdf', 'page': 0}


# Step-3: Creating embeddings

In [15]:
from langchain_openai import OpenAIEmbeddings

In [16]:
embeddings = OpenAIEmbeddings()


In [17]:
# A sample embedding

sample_embedding = embeddings.embed_query("You must follow the rules")
print(sample_embedding)

[-0.00956281978366329, -0.012191951005027816, 0.00579310985047816, -0.03309612155710863, -0.02786363439935308, 0.03072474691076965, -0.033482758282292205, -0.017514652591513257, 0.0022360502092762602, -0.017140903508816432, 0.030338110185586076, -0.01316498757055276, 0.012720354591533548, -0.002147445804534586, -0.01908697663986632, -0.01393181719967654, 0.033482758282292205, -0.012017964106166156, 0.016058321050831464, 0.006598603338384822, 0.00025393186583473837, 0.016200086980830995, -0.012378825546376229, -0.03783886712449608, -0.012469040906428747, -0.01623874990829125, -0.012900785311638583, -0.008873316940782652, 0.008029160525415732, -0.016161424053370736, 0.019151414852300083, -0.0055321299678469825, -0.0024986410472182816, -0.006888581347933817, -0.022747139749268806, 0.010039671868899385, -0.0032461392126119314, 0.0011872975799883628, -0.0006073421429670122, -0.011083591399424095, 0.03330232756218717, 0.009491936818663525, -0.0005070581958063906, -0.00615719320130992, -0.019

In [18]:
len(sample_embedding)

1536

In [19]:
sample_docs=["You must follow the rules",
            "You must not disclose the rules"]

embeded_vectors1= embeddings.embed_documents(sample_docs)
print(embeded_vectors1)

[[-0.00956281978366329, -0.012191951005027816, 0.00579310985047816, -0.03309612155710863, -0.02786363439935308, 0.03072474691076965, -0.033482758282292205, -0.017514652591513257, 0.0022360502092762602, -0.017140903508816432, 0.030338110185586076, -0.01316498757055276, 0.012720354591533548, -0.002147445804534586, -0.01908697663986632, -0.01393181719967654, 0.033482758282292205, -0.012017964106166156, 0.016058321050831464, 0.006598603338384822, 0.00025393186583473837, 0.016200086980830995, -0.012378825546376229, -0.03783886712449608, -0.012469040906428747, -0.01623874990829125, -0.012900785311638583, -0.008873316940782652, 0.008029160525415732, -0.016161424053370736, 0.019151414852300083, -0.0055321299678469825, -0.0024986410472182816, -0.006888581347933817, -0.022747139749268806, 0.010039671868899385, -0.0032461392126119314, 0.0011872975799883628, -0.0006073421429670122, -0.011083591399424095, 0.03330232756218717, 0.009491936818663525, -0.0005070581958063906, -0.00615719320130992, -0.01

In [20]:
print(len(embeded_vectors1))
print(len(embeded_vectors1[0]))

2
1536


In [21]:
print(sample_embedding)
print(embeded_vectors1[0])

[-0.00956281978366329, -0.012191951005027816, 0.00579310985047816, -0.03309612155710863, -0.02786363439935308, 0.03072474691076965, -0.033482758282292205, -0.017514652591513257, 0.0022360502092762602, -0.017140903508816432, 0.030338110185586076, -0.01316498757055276, 0.012720354591533548, -0.002147445804534586, -0.01908697663986632, -0.01393181719967654, 0.033482758282292205, -0.012017964106166156, 0.016058321050831464, 0.006598603338384822, 0.00025393186583473837, 0.016200086980830995, -0.012378825546376229, -0.03783886712449608, -0.012469040906428747, -0.01623874990829125, -0.012900785311638583, -0.008873316940782652, 0.008029160525415732, -0.016161424053370736, 0.019151414852300083, -0.0055321299678469825, -0.0024986410472182816, -0.006888581347933817, -0.022747139749268806, 0.010039671868899385, -0.0032461392126119314, 0.0011872975799883628, -0.0006073421429670122, -0.011083591399424095, 0.03330232756218717, 0.009491936818663525, -0.0005070581958063906, -0.00615719320130992, -0.019

# Step-4: Storing in Vector Stores

In [22]:
from langchain.vectorstores import Chroma

In [23]:
emp_rules_db= Chroma.from_documents(chunks,
                                    embeddings,
                                    persist_directory="emp_rules_db"
                          )


# Step-5: Retrieval

In [25]:
retriever = emp_rules_db.as_retriever()
result=retriever.invoke("What is the policy for sick leaves",
                                        top_k=3)
result

[Document(page_content='sick days and personal absence days as authorized by the Company for its other employees.\nARTICLE 6\nFringe Benefits\nEmployee shall be entitled to participate  in and receive benefits from all of the Company’ s employee\nbenefit plans that are now , or in the future may be, maintained by the', metadata={'page': 1, 'source': 'data/EMPLOYEE_AGREEMENT.pdf'}),
 Document(page_content='5.2. Holidays.    Employee shall be entitled to the same paid holidays as authorized by the Company\nfor its other employees.\n5.3. Sick Days  and Personal Absence Days.   Employee shall be entitled to the same number of paid', metadata={'page': 1, 'source': 'data/EMPLOYEE_AGREEMENT.pdf'}),
 Document(page_content='ARTICLE 5\nVacations and Other  Paid Absences\n5.1. Vacation Days.   Employee shall be entitled to the same paid vacation days each calendar year\nduring the term of this Employee Agreement as authorized by the Company for its other employees.', metadata={'page': 1, 'source'

In [26]:
for i in range(len(result)):
  print(result[i].metadata)

{'page': 1, 'source': 'data/EMPLOYEE_AGREEMENT.pdf'}
{'page': 1, 'source': 'data/EMPLOYEE_AGREEMENT.pdf'}
{'page': 1, 'source': 'data/EMPLOYEE_AGREEMENT.pdf'}
{'page': 11, 'source': 'data/EMPLOYEE_AGREEMENT.pdf'}


You can also set a retrieval method that sets a similarity score threshold and only returns documents with a score above that threshold.


### pretty_print_docs

In [28]:
retriever = emp_rules_db.as_retriever()
result=retriever.get_relevant_documents("What is the policy for insurance?",
                                        top_k=3)


In [29]:
for i in range(len(result)):
  print(result[i].metadata)

{'page': 11, 'source': 'data/EMPLOYEE_AGREEMENT.pdf'}
{'page': 2, 'source': 'data/EMPLOYEE_AGREEMENT.pdf'}
{'page': 6, 'source': 'data/EMPLOYEE_AGREEMENT.pdf'}
{'page': 2, 'source': 'data/EMPLOYEE_AGREEMENT.pdf'}


In [31]:
retriever = emp_rules_db.as_retriever()
result=retriever.invoke("Employee Monthly Salary ",
                                        top_k=3)
result

[Document(page_content='Compensation of Employee\n4.1. Base Com pensation.   For all services rendered by Employee under this Employee Agreement, the\nCompany agrees to pay Employee the rate of $14,583 per month (the “base salary”), which shall be payable', metadata={'page': 1, 'source': 'data/EMPLOYEE_AGREEMENT.pdf'}),
 Document(page_content='agreed upon by Employee.', metadata={'page': 11, 'source': 'data/EMPLOYEE_AGREEMENT.pdf'}),
 Document(page_content='to Employ ee not less frequently than bi-monthly , or as is consistent with the Company’ s practice for its other\nemployees.  \n4.2. Other Compensation.   Employee shall receive other compensation as more fully described on\nAppendix B , attached hereto and made a part hereof.', metadata={'page': 1, 'source': 'data/EMPLOYEE_AGREEMENT.pdf'}),
 Document(page_content='ARTICLE 3\nPlace of Employment\n3.1. Place of Employment.    Employee shall perform his duties under this Employee Agreement at\n802 NW  5th Avenue, Suite 100, Gainesvil

In [32]:
for i in range(len(result)):
  print(result[i].metadata)

{'page': 1, 'source': 'data/EMPLOYEE_AGREEMENT.pdf'}
{'page': 11, 'source': 'data/EMPLOYEE_AGREEMENT.pdf'}
{'page': 1, 'source': 'data/EMPLOYEE_AGREEMENT.pdf'}
{'page': 1, 'source': 'data/EMPLOYEE_AGREEMENT.pdf'}


# RAG Example-2

In [ ]:
# Step-1:Document Loading
from langchain.document_loaders import PyPDFLoader
!wget https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Banking_System_Doc/BASEL.pdf
loader = PyPDFLoader("BASEL.pdf")
pages = loader.load()
print(len(pages))

In [ ]:
#Step-2:Split the data into Chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(pages)
print(len(chunks))

In [ ]:
#Step-3: Creating Embeddings
from langchain.embeddings import OpenAIEmbeddings, CohereEmbeddings
#embeddings = OpenAIEmbeddings()
embeddings = CohereEmbeddings()

#Step-4: Storing in a Vector DB
#!rm -rf basel_norms_db
basel_norms_db= Chroma.from_documents(chunks,
                                    embeddings,
                                    persist_directory="basel_norms_db"
                          )
basel_norms_db.persist()

In [ ]:
#Step-5 Retrieval

retriever = basel_norms_db.as_retriever()
result=retriever.get_relevant_documents("What percentage is the minimum Capital Requirements",
                                        top_k=3)
print(result)
print(len(result))
print([i.metadata for i in result])

In [ ]:
retriever = basel_norms_db.as_retriever()
result=retriever.get_relevant_documents("What are PD and LGD",
                                        top_k=3)
print(result)
print(len(result))
print([i.metadata for i in result])

# MultiQueryRetriever

In [ ]:
!pip install wikipedia -q

In [ ]:
from langchain.document_loaders import wikipedia

#Document Loading
loader=wikipedia.WikipediaLoader(query="MS Dhoni")
documents=loader.load()

#Splitting
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
docs=text_splitter.split_documents(documents)
len(docs)

#Embeddings and VectorDB
from langchain.embeddings import OpenAIEmbeddings
#embeddings=OpenAIEmbeddings()
embeddings=CohereEmbeddings()

embeddings_db=Chroma.from_documents(docs,embeddings,
                                    persist_directory="wiki_db")
embeddings_db.persist()



In [ ]:
from langchain.retrievers import MultiQueryRetriever
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI , Cohere


llm=OpenAI(temperature=0)
#llm=Cohere(temperature=0)

In [ ]:
llm_based_retriver=MultiQueryRetriever.from_llm(
    retriever=embeddings_db.as_retriever(),
    llm=llm
)

In [ ]:
llm_based_retriver

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [ ]:
question1="What is the DOB of Dhoni?"
question2= "What Sport does Dhoni Play?"
rel_docs1=llm_based_retriver.get_relevant_documents(question1)
rel_docs2=llm_based_retriver.get_relevant_documents(question2)

In [ ]:
rel_docs1

In [ ]:
rel_docs2

# Contextual compression

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [ ]:
#llm=OpenAI(temperature=0)
llm=Cohere(temperature=0)

compressor=LLMChainExtractor.from_llm(llm)

compression_retriever=ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=embeddings_db.as_retriever()
)

In [ ]:
compressed_docs=compression_retriever.get_relevant_documents(question1)

In [ ]:
compressed_docs[0].metadata

In [ ]:
compressed_docs[0].metadata["summary"]

# RetrievalQA Chain

In [ ]:
question1="What is the DOB of Dhoni?"
question2= "What Sport does Dhoni Play?"
rel_docs1=llm_based_retriver.get_relevant_documents(question1)
rel_docs2=llm_based_retriver.get_relevant_documents(question2)

In [ ]:
from langchain.chains import RetrievalQA
#llm=OpenAI(temperature=0)
llm=Cohere(temperature=0)

Q_AChain=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",# It takes a list of documents, inserts them all into a prompt
    retriever=llm_based_retriver
)

In [ ]:
query="What is the DOB of Dhoni?"
docs=Q_AChain({"query":query})
docs["result"]

In [ ]:
docs

In [ ]:
print(Q_AChain.combine_documents_chain.llm_chain.prompt.template)

#Validation of the Results

In [38]:
#COI Data

from langchain.document_loaders import PyPDFLoader

#Document Loading
loader = PyPDFLoader("data/COI.pdf")
pages = loader.load()

full_text =""
for page in pages:
  full_text += page.page_content

print("Pages", len(pages))
print("Lines" , len(full_text.split("\n")))
print("Words" , len(full_text.split(" ")))
print("Charecters", len(full_text))

#Split the data into Chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=400)
chunks = text_splitter.split_documents(pages)
print(len(chunks))

#Embeddings and Vector DB
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
embeddings = OpenAIEmbeddings()

coi_db= Chroma.from_documents(chunks,
                             embeddings,
                             persist_directory="vectorstores/coi_db"
                             )
coi_db.persist()

#Retrieval Q_A Chain

from langchain.chains import RetrievalQA
from langchain_openai import OpenAI 

llm=OpenAI(temperature=0)

Q_AChain=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",# It takes a list of documents, inserts them all into a prompt
    retriever=coi_db.as_retriever()
)

Pages 402
Lines 14053
Words 172539
Charecters 869095
687


In [35]:
query="""
According to constitution of India what are the fundamental rights of citizens of India ?
"""
docs=Q_AChain({"query":query})
print(docs["result"])

c:\Users\WIN-10\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


 The fundamental rights of citizens of India are outlined in Part III of the Constitution, and include the right to equality before the law, prohibition of discrimination on grounds of religion, race, caste, sex or place of birth, protection of life and personal liberty, freedom of speech and expression, freedom of religion, and the right to education.


## RAG Validation Process

In [36]:
llm=OpenAI(temperature=0)

Q_AChain=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",# It takes a list of documents, inserts them all into a prompt
    retriever=coi_db.as_retriever(),
    input_key="question" # Add this extra parameter to access the user questions using input_key
)

##Get Validation data

In [ ]:
!wget https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/COI/COI_Q_A.csv

In [ ]:
import pandas as pd
test_data=pd.read_csv("COI_Q_A.csv")
test_data.head()

#Creating a iterative "Question", "Answer" pairs
test_qa_pairs = []
for index, row in test_data.iterrows():
  question = row['Question']
  answer = row['Answer']
  test_qa_pairs.append({'question': question, 'answer': answer})
print(test_qa_pairs)

In [ ]:
#Add one more Question and Answer
question_new="Who is Narendra Modi?"
answer_new="""
Narendra Modi is the current Prime Minister of India, serving since 2014.
He is a member of the Bharatiya Janata Party (BJP) and
previously served as the Chief Minister of Gujarat from 2001 to 2014.
"""
test_qa_pairs.append({'question': question_new, 'answer': answer_new})

## Get the Predictions

In [ ]:
predictions = Q_AChain.apply(test_qa_pairs)
predictions

## Compare Actual Answers with predicted Answers

In [ ]:
#The comparison is done using an LLM
from langchain.evaluation.qa import QAEvalChain

llm=OpenAI(temperature=0)

qa_eval_chain = QAEvalChain.from_llm(llm)

In [ ]:
eval_result = qa_eval_chain.evaluate(test_qa_pairs,
                                     predictions,
                                      question_key="question",
                                      answer_key="answer"
                                     )
print(eval_result)

## Calculate The Accuracy

In [ ]:
# Initialize counters
correct_count = 0
incorrect_count = 0

# Count occurrences
for item in eval_result:
    if item['results'].strip() == 'CORRECT':
        correct_count += 1
    elif item['results'].strip() == 'INCORRECT':
        incorrect_count += 1

# Calculate percentages
total_count = len(eval_result)
correct_percentage = (correct_count / total_count) * 100
incorrect_percentage = (incorrect_count / total_count) * 100

# Print the results
print(f"CORRECT: {correct_percentage}%")
print(f"INCORRECT: {incorrect_percentage}%")


In [40]:
llm.invoke("how many  t20 world cups won by india  ")

'\n\nIndia has won one T20 World Cup, in 2007.'

In [41]:
llm.invoke("give me some playrs from the team?  ")

'\n\n1. LeBron James\n2. Anthony Davis\n3. Kyle Kuzma\n4. Rajon Rondo\n5. Danny Green\n6. Kentavious Caldwell-Pope\n7. Dwight Howard\n8. Alex Caruso\n9. Markieff Morris\n10. Talen Horton-Tucker'